In [1079]:
'''
Name: Aman Kumar
Hawk ID: A20538809
Assignment 2
CS 584
'''

'\nName: Aman Kumar\nHawk ID: A20538809\nAssignment 2\nCS 584\n'

In [1080]:
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.model_selection import KFold
from sklearn.utils import resample
import csv

# Model to string method

In [1081]:
def model_to_string(coeffs, m, n):
    ones = f"{coeffs[0]} "
    cosines = [f"{coeffs[j]:+} cos(2π * {j})" for j in range(1, m + 1)]
    sines = [f"{coeffs[j]:+} sin(2π * {j - m})" for j in range(m + 1, m + n + 1)]
    return ones + " ".join(cosines) + " " + " ".join(sines)

# Fitting the model

In [1082]:
#method to fit model
def fit_model(data, m, n):
    x_sample = data['x'].values.reshape(-1, 1)
    y_sample = data['y'].values.reshape(-1, 1)
    ones = np.ones_like(x_sample)
    cosines = np.array([np.cos(2 * np.pi * j * x_sample) for j in range(1, m + 1)])[:, :, 0].T
    sines = np.array([np.sin(2 * np.pi * j * x_sample) for j in range(1, n + 1)])[:, :, 0].T
    dmatrix = np.concatenate([ones, cosines, sines], axis=1)

    u, s, vT = np.linalg.svd(dmatrix, full_matrices=False)
    uT = np.transpose(u)
    v = np.transpose(vT)
    s_inv = np.power(s, -1)
    p_inv = np.dot(v, np.dot(np.diag(s_inv), uT))
    coeffs = np.dot(p_inv, y_sample)
    model_stringified = model_to_string(coeffs.reshape(-1), m, n)

    outputs = np.dot(dmatrix, coeffs)
    resids = y_sample - outputs
    rmse = np.sqrt(np.mean(np.square(resids.reshape(-1))))

    return coeffs


# Generating the data

In [1083]:
#method to generate data
def generate_data(sample_size, noise_std, file_name):
    rng = np.random.default_rng()
    x_sample = rng.uniform(-10, 10, sample_size)
    noise = rng.normal(0, noise_std, sample_size)
    offset = rng.uniform(1)
    y_sample = signal.sawtooth(2 * np.pi * x_sample + offset) + noise
    
    with open(file_name, "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["x", "y"])
        for x, y in zip(x_sample, y_sample):
            writer.writerow([x, y])
    
    data = pd.DataFrame({'x': x_sample, 'y': y_sample})
    return data


# Log likelihood method

In [1084]:
#method to calculate log likelihood 
#m is the number of cosine terms
#n is the number of sine terms


def calculate_log_likelihood(data, coeffs, m, n):
    
    #extract the input features 
    x_in = data['x'].values.reshape(-1, 1)
    
    #extract the target variables
    y_in = data['y'].values.reshape(-1, 1)
    
    ones = np.ones_like(x_in)
    cosines = np.array([np.cos(2 * np.pi * i * x_in) for i in range(1, m + 1)])[:, :, 0].T
    sines = np.array([np.sin(2 * np.pi * i * x_in) for i in range(1, n + 1)])[:, :, 0].T
    
    #datrix matrix of intercepts, sines and cosines
    dmatrix = np.concatenate([ones, cosines, sines], axis=1)
    
    op = np.dot(dmatrix, coeffs)
    rem = y_in - op
    rss = np.sum(rem**2)
    
    n_samples = len(data)
    
    #number of parameters
    num_params = m + n + 1 
    deg = n_samples - num_params 
    
    #calculation of log likelihood
    log_likelihood = -0.5 * n_samples * np.log(2 * np.pi * rss / deg)  # assuming normal distribution
    
    return log_likelihood

# AIC Calculation

In [1085]:
#calculate AIC by using log likelihood


def calculate_aic(data, coeffs, m, n):
    #k is the number of parameters
    k = m + n +1
    
    #AIC is calculated using the formula 
    #AIC = 2*k - 2*log(L)
    #where L is the likelihood value 
    #this is calculated by calculating the likelihood function
    aic = 2*k - 2*np.log(calculate_log_likelihood(data, coeffs, m, n))
    return aic

# RMSE's calculation

In [1086]:
#calculating the root mean square error

def calculate_rmse(data, coeffs, m, n):
    
    #x is the input features
    x_sample = data['x'].values.reshape(-1, 1)
    
    #y is the target features
    y_sample = data['y'].values.reshape(-1, 1)
    
    #ones cosines and sines values
    ones = np.ones_like(x_sample)
    cosines = np.array([np.cos(2 * np.pi * i * x_sample) for i in range(1, m + 1)])[:, :, 0].T
    sines = np.array([np.sin(2 * np.pi * i * x_sample) for i in range(1, n + 1)])[:, :, 0].T
    
    #dmatrix of sines, cosines and intercept
    dmatrix = np.concatenate([ones, cosines, sines], axis=1)

    outputs = np.dot(dmatrix, coeffs)
    resids = y_sample - outputs
    
    #rmse is the sqrt of mean of (y_sample - outputs)^2
    rmse = np.sqrt(np.mean(np.square(resids.reshape(-1))))
    return rmse

# K_fold_cross_validation

In [1087]:
#method to calculate the k fold cross validation

def k_fold_cross_validation(data, m, n, k):
    
    #k number of splits
    kfc = KFold(n_splits=k)
    rmses = []
    
    #iterate over each fold and split the data into the training and test sets
    for train_i, test_i in kfc.split(data):
        train_data = data.iloc[train_i]
        test_data = data.iloc[test_i]
        
        #fits the regression model with the training data and computing the coefficients
        coeffs = fit_model(train_data, m, n)
        
        #calculates root mean square error on the testing data by calling the calculate_rmse function
        rmse = calculate_rmse(test_data, coeffs, m, n)
        rmses.append(rmse)
        
    #calculating the mean of the rmses values 
    return np.mean(rmses)

# Bootstrap Resampling

In [1088]:
#method to compute bootstrap resampling to evaluate the performance of the model
#num_bootstraps is the number of bootstrap samples to generate

def bootstrap(data, m, n, num_bs):
    rmses = []
    
    #iterate throught the num_bootstraps to generate multiple bootstrap samples
    for _ in range(num_bs):
        
        #bs_data is the bootstrap data
        bs_data = resample(data, replace=True)
        
        #compute coefficients as per the bootstrap data
        coeffs = fit_model(bs_data, m, n)
        
        #calculate rmse based on the data, coeffs and the parameter values
        rmse = calculate_rmse(data, coeffs, m, n)
        rmses.append(rmse)
        
    return np.mean(rmses)

# Bias variance computation

In [1089]:
#method to calculate the bias and variance of the model for different values of m and n

def calculate_bias_variance(data, coeffs, m, n):
    #extract the input features and the target variables from the data
    x_in = data['x'].values.reshape(-1, 1)
    y_in = data['y'].values.reshape(-1, 1)
    
    #intercepts, sines and cosines values for the dmatrix
    ones = np.ones_like(x_in)
    cos = np.array([np.cos(2 * np.pi * i * x_in) for i in range(1, m + 1)])[:, :, 0].T
    sine = np.array([np.sin(2 * np.pi * i * x_in) for i in range(1, n + 1)])[:, :, 0].T
    
    #matrix for the sine, cosine and intercept terms
    dmatrix = np.concatenate([ones, cos, sine], axis=1)
    
    # predicted value of target values
    y_predicted = np.dot(dmatrix, coeffs)
    
    #calculate bias from the sample and the mean of predicted values
    bias = np.mean((y_in - np.mean(y_predicted)) ** 2)
    
    #calculate variance from the difference of predicted and the mean of the predicted values
    variance = np.mean((y_predicted - np.mean(y_predicted)) ** 2)
    
    return bias, variance

In [1090]:
def main():
    input_file = "sample.csv"

    data = pd.read_csv(input_file)
    
    #noise values from 0.01 to 0.1
    noise_stds = [0.01, 0.05, 0.1]
    
    #range of values of m 
    m_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    
    #n values 
    n_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    
    #variables to determine the value of m and n for min aic
    min_aic = np.inf
    best_aic_m = None
    best_aic_n = None
    
    #variables to determine the value of m and n for min values of k_5_fold rmse
    min_k_5_fold_rmse = np.inf
    best_k_5_fold_m = None
    best_k_5_fold_n = None
    best_k_5_fold = None
    
    #variables to determine the value of m and n for min values of k_10_fold rmse
    min_k_10_fold_rmse = np.inf
    best_k_10_fold_m = None
    best_k_10_fold_n = None
    best_k_10_fold = None
    
    #variables to determine the value of m and n for min values of bootstrap samples
    best_rmse = np.inf
    best_bs_m = None
    best_bs_n = None
    
    

    #for size in sample_sizes:
    
    #iterating through each value of noise passed
    for noise in noise_stds:
        #iterating through each value of parameter m
        for m in m_values:
            #iterarting through each value of parameter n
            for n in n_values:
                print(f"Noise Level: {noise}, m: {m}, n: {n}")
                
                #generate the data by calling the generate data function
                synthetic_data = generate_data(len(data), noise, 'sample.csv')
                
                #compute coefficients from the fit model function
                coeffs = fit_model(synthetic_data, m, n)
                model_stringified = model_to_string(coeffs.reshape(-1), m, n)
                #this will print the model coefficients
                print("Model:", model_stringified)
                
                #compute rmse by calling the calculate_rmse function
                rmse = calculate_rmse(synthetic_data, coeffs, m, n)
                print("RMSE:", rmse)
                
                #calculation of AIC 
                aic = calculate_aic(synthetic_data, coeffs, m, n)
                print("AIC:", aic)
                if aic < min_aic:
                    min_aic = aic
                    best_aic_m = m
                    best_aic_n = n
                
                #computation of k_fold_rmse for k = 5
                k_fold_rmse_5 = k_fold_cross_validation(synthetic_data, m, n, 5)
                print("5-Fold Cross Validation RMSE:", k_fold_rmse_5)
                
                #computation of k_fold_rmse for k = 10
                k_fold_rmse_10 = k_fold_cross_validation(synthetic_data, m, n, 10)
                print("10-Fold Cross Validation RMSE:", k_fold_rmse_10)
                
                #computing value of m and n for min value of k_fold rmse for k = 5
                if k_fold_rmse_5 < min_k_5_fold_rmse:
                    min_k_5_fold_rmse = k_fold_rmse_5
                    best_k_5_fold_m = m
                    best_k_5_fold_n = n
                
                #computing value of m and n for min value of k_fold rmse for k = 10
                if k_fold_rmse_10 < min_k_10_fold_rmse:
                    min_k_10_fold_rmse = k_fold_rmse_10
                    best_k_10_fold_m = m
                    best_k_10_fold_n = n
                
                #computation of bootstrap below
                #number of bootstrap samples that we are taking
                num_bootstraps = 100
                bootstrap_rmse = bootstrap(synthetic_data, m, n, num_bootstraps)
                print("Bootstrap RMSE:", bootstrap_rmse)
                
                #computing value of m and n for min value of bootstrap rmse
                if bootstrap_rmse < best_rmse:
                    best_rmse = bootstrap_rmse
                    best_bs_m = m
                    best_bs_n = n
                    
                
                #computing the bias and variance of the model
                bias, variance = calculate_bias_variance(synthetic_data, coeffs, m, n)
                print(f"Bias: {bias}")
                print(f"Variance: {variance}")
                    
    #values of m and n for each method      
    print(f"Minimum AIC: {min_aic}, m: {best_aic_m}, n: {best_aic_n}")
    print(f"Minimum k-5_Fold Cross Validation RMSE: {min_k_5_fold_rmse}, m: {best_k_5_fold_m}, n: {best_k_5_fold_n}")
    print(f"Minimum k-10_Fold Cross Validation RMSE: {min_k_10_fold_rmse}, m: {best_k_10_fold_m}, n: {best_k_10_fold_n}")
    print(f"Minimum Bootsrap value: {best_rmse}, m: {best_bs_m}, n: {best_bs_n}")
    print()



    

if __name__ == "__main__":
    main()

Noise Level: 0.01, m: 1, n: 1
Model: 0.002243584507136347 -0.5336864864730149 cos(2π * 1) -0.35199584436405473 sin(2π * 1)
RMSE: 0.3603168000782852
AIC: -7.848891993092899
5-Fold Cross Validation RMSE: 0.36039178954998785
10-Fold Cross Validation RMSE: 0.3603069306641544
Bootstrap RMSE: 0.3603705309885265
Bias: 0.3340677322525515
Variance: 0.20423953583389667
Noise Level: 0.01, m: 1, n: 2
Model: 0.0030696315151560833 -0.5276170913690302 cos(2π * 1) -0.34736837851031555 sin(2π * 1) +0.1249613939132629 sin(2π * 2)
RMSE: 0.3532359743080325
AIC: -6.204675882357863
5-Fold Cross Validation RMSE: 0.3532049015978621
10-Fold Cross Validation RMSE: 0.3531739821862119
Bootstrap RMSE: 0.35331633146730723
Bias: 0.3319185408002275
Variance: 0.20714288725488195
Noise Level: 0.01, m: 1, n: 3
Model: -0.007108099822300593 -0.5412285306847244 cos(2π * 1) -0.3309792368308564 sin(2π * 1) +0.14225335928042238 sin(2π * 2) +0.20920036621837357 sin(2π * 3)
RMSE: 0.31567844246287663
AIC: -5.514466450273053
5-Fo

Bootstrap RMSE: 0.20959626172670917
Bias: 0.33140511348181323
Variance: 0.28751465745628774
Noise Level: 0.01, m: 2, n: 8
Model: -0.0037004108158813203 -0.539451436292488 cos(2π * 1) -0.28636847354132106 cos(2π * 2) -0.33974595426280874 sin(2π * 1) +0.13915394362138211 sin(2π * 2) +0.21232285091855546 sin(2π * 3) +0.09779791541422331 sin(2π * 4) -0.042669028346078025 sin(2π * 5) -0.10179318324114961 sin(2π * 6) -0.06537833069686273 sin(2π * 7) +0.017311758926427466 sin(2π * 8)
RMSE: 0.21054469098745193
AIC: 4.476371897479368
5-Fold Cross Validation RMSE: 0.21070742794192973
10-Fold Cross Validation RMSE: 0.21050624594250006
Bootstrap RMSE: 0.21064584366438097
Bias: 0.3346226724139239
Variance: 0.2902936055109229
Noise Level: 0.01, m: 2, n: 9
Model: -0.001721582525322246 -0.5389194837404372 cos(2π * 1) -0.2872534097550805 cos(2π * 2) -0.3413703107056745 sin(2π * 1) +0.1352510080343763 sin(2π * 2) +0.2109665032357556 sin(2π * 3) +0.10151367125159236 sin(2π * 4) -0.03944586931597477 sin(2

Bootstrap RMSE: 0.28170781356211366
Bias: 0.3362883846841959
Variance: 0.256970667189288
Noise Level: 0.01, m: 4, n: 2
Model: 0.00027650082598487755 -0.5322828420886773 cos(2π * 1) -0.29495764175279265 cos(2π * 2) -0.03665471325903219 cos(2π * 3) +0.12265419141823733 cos(2π * 4) -0.34993102397318876 sin(2π * 1) +0.1286377165188046 sin(2π * 2)
RMSE: 0.2749841611026469
AIC: -2.4416301593029957
5-Fold Cross Validation RMSE: 0.27513763479898246
10-Fold Cross Validation RMSE: 0.274955533932258
Bootstrap RMSE: 0.27506942107379767
Bias: 0.33818700013162833
Variance: 0.26257071127430104
Noise Level: 0.01, m: 4, n: 3
Model: 0.0008938854364584628 -0.5343718863435366 cos(2π * 1) -0.29147320251117737 cos(2π * 2) -0.029175251311437757 cos(2π * 3) +0.1236079209299902 cos(2π * 4) -0.34571176227773287 sin(2π * 1) +0.1322186396913515 sin(2π * 2) +0.2110817806436817 sin(2π * 3)
RMSE: 0.22304335421368265
AIC: -1.3348452536538673
5-Fold Cross Validation RMSE: 0.22301610652380605
10-Fold Cross Validation R

Bootstrap RMSE: 0.18952731449208238
Bias: 0.3308534772844482
Variance: 0.2949793957430906
Noise Level: 0.01, m: 5, n: 6
Model: 0.002690850316052624 -0.5337113232252733 cos(2π * 1) -0.2934881211029986 cos(2π * 2) -0.034468830775693356 cos(2π * 3) +0.12019190727723988 cos(2π * 4) +0.12575404417440253 cos(2π * 5) -0.34888634916637357 sin(2π * 1) +0.12938307848714425 sin(2π * 2) +0.21219293943663772 sin(2π * 3) +0.10908365774386158 sin(2π * 4) -0.03312400536008578 sin(2π * 5) -0.10340753039166994 sin(2π * 6)
RMSE: 0.1854118972045244
AIC: 6.113414618905519
5-Fold Cross Validation RMSE: 0.18570156850409977
10-Fold Cross Validation RMSE: 0.18541624943649476
Bootstrap RMSE: 0.18552667043686402
Bias: 0.3356942491173072
Variance: 0.30131667749232705
Noise Level: 0.01, m: 5, n: 7
Model: -0.0010043950880860922 -0.5356826744134464 cos(2π * 1) -0.28827278270128054 cos(2π * 2) -0.027121846377259028 cos(2π * 3) +0.12141808501186757 cos(2π * 4) +0.12174570700089121 cos(2π * 5) -0.34434849600182893 sin(

Bootstrap RMSE: 0.17212601263155874
Bias: 0.33701212985709694
Variance: 0.307431746928901
Noise Level: 0.01, m: 6, n: 9
Model: -0.00027014785781464493 -0.5362347053324994 cos(2π * 1) -0.29012478757726 cos(2π * 2) -0.02954238664913215 cos(2π * 3) +0.12059497547001183 cos(2π * 4) +0.12281010005496454 cos(2π * 5) +0.03017942994439215 cos(2π * 6) -0.34545344517646304 sin(2π * 1) +0.13140968563160454 sin(2π * 2) +0.2090599984006937 sin(2π * 3) +0.1048934616974595 sin(2π * 4) -0.0357838055941458 sin(2π * 5) -0.103352111207544 sin(2π * 6) -0.06895103264114497 sin(2π * 7) +0.010403408513102603 sin(2π * 8) +0.06628799311135256 sin(2π * 9)
RMSE: 0.16487446497639907
AIC: 13.828560561040007
5-Fold Cross Validation RMSE: 0.1650573246807797
10-Fold Cross Validation RMSE: 0.16476651075001097
Bootstrap RMSE: 0.16498990398508673
Bias: 0.3304431904795041
Variance: 0.3032596012782513
Noise Level: 0.01, m: 6, n: 10
Model: -0.002766663677565444 -0.5391763469844028 cos(2π * 1) -0.28751243213141153 cos(2π * 

...

Bootstrap RMSE: 0.21828674360078243
Bias: 0.3443219658098987
Variance: 0.29672555720123883
Noise Level: 0.1, m: 6, n: 5
Model: 0.0009577408051937378 -0.5352630096135436 cos(2π * 1) -0.285487295374853 cos(2π * 2) -0.02562422444880698 cos(2π * 3) +0.11848238391292512 cos(2π * 4) +0.11335073232615274 cos(2π * 5) +0.021707164318230963 cos(2π * 6) -0.3438009817618465 sin(2π * 1) +0.13244724688663076 sin(2π * 2) +0.20633956998658642 sin(2π * 3) +0.09973188828970478 sin(2π * 4) -0.03625835292713166 sin(2π * 5)
RMSE: 0.2112626691755275
AIC: 6.487220325319051
5-Fold Cross Validation RMSE: 0.21135685250715927
10-Fold Cross Validation RMSE: 0.21136982983126407
Bootstrap RMSE: 0.2114029188439356
Bias: 0.34103345243717165
Variance: 0.2964015370500041
Noise Level: 0.1, m: 6, n: 6
Model: -0.000720702446237145 -0.5343142814514642 cos(2π * 1) -0.28351729224239236 cos(2π * 2) -0.026944510983098087 cos(2π * 3) +0.11857266721320614 cos(2π * 4) +0.11940484324483397 cos(2π * 5) +0.02801790809385348 cos(2π *

Bootstrap RMSE: 0.19804323807958826
Bias: 0.3426688714622959
Variance: 0.3035029677655183
Noise Level: 0.1, m: 7, n: 7
Model: 0.004949455337452656 -0.5300609279646213 cos(2π * 1) -0.2928655096819769 cos(2π * 2) -0.03244322999508212 cos(2π * 3) +0.11685840861262183 cos(2π * 4) +0.12389566368189456 cos(2π * 5) +0.03273482492014159 cos(2π * 6) -0.058025836538656006 cos(2π * 7) -0.3487253510061815 sin(2π * 1) +0.1271204282450083 sin(2π * 2) +0.20533326277341446 sin(2π * 3) +0.10705906895573114 sin(2π * 4) -0.03573333901161536 sin(2π * 5) -0.10052238104827718 sin(2π * 6) -0.07186744660192626 sin(2π * 7)
RMSE: 0.1936200678026643
AIC: 12.230313227050633
5-Fold Cross Validation RMSE: 0.19373399256107604
10-Fold Cross Validation RMSE: 0.19342854209905197
Bootstrap RMSE: 0.1937571318521325
Bias: 0.3405578968662342
Variance: 0.3030691662103246
Noise Level: 0.1, m: 7, n: 8
Model: -0.002327622823428024 -0.5405832470906315 cos(2π * 1) -0.28780885218316954 cos(2π * 2) -0.027704914774252833 cos(2π * 3

Bootstrap RMSE: 0.18418229668828545
Bias: 0.3407976417659902
Variance: 0.3069289024978098
Noise Level: 0.1, m: 8, n: 9
Model: -0.0025412228834665743 -0.5371788828486654 cos(2π * 1) -0.2870416071971237 cos(2π * 2) -0.028213530622013365 cos(2π * 3) +0.12164193924646688 cos(2π * 4) +0.119897095983765 cos(2π * 5) +0.02776554772526557 cos(2π * 6) -0.059300037230839035 cos(2π * 7) -0.07979468356647099 cos(2π * 8) -0.34014100564989347 sin(2π * 1) +0.13484546391865554 sin(2π * 2) +0.21012817484131907 sin(2π * 3) +0.10399606267719971 sin(2π * 4) -0.03801080658751727 sin(2π * 5) -0.100939448271191 sin(2π * 6) -0.06883953809505489 sin(2π * 7) +0.011423557002677998 sin(2π * 8) +0.06610979435961295 sin(2π * 9)
RMSE: 0.1791498846701287
AIC: 18.026373668811228
5-Fold Cross Validation RMSE: 0.1791390888312313
10-Fold Cross Validation RMSE: 0.17910366350666518
Bootstrap RMSE: 0.1793254512161993
Bias: 0.34171275501402193
Variance: 0.30961807383670187
Noise Level: 0.1, m: 8, n: 10
Model: -0.0013685040590

Bootstrap RMSE: 0.17266554548089427
Bias: 0.3377755596289314
Variance: 0.3080252558046206
Noise Level: 0.1, m: 9, n: 10
Model: 0.001505571958954787 -0.5329708754325075 cos(2π * 1) -0.29025901244369734 cos(2π * 2) -0.03033822921848226 cos(2π * 3) +0.12052957824098057 cos(2π * 4) +0.12593524051970867 cos(2π * 5) +0.03197623610106501 cos(2π * 6) -0.06029392844789552 cos(2π * 7) -0.07933006896664616 cos(2π * 8) -0.03158873125402013 cos(2π * 9) -0.3456923556521234 sin(2π * 1) +0.1316371071178984 sin(2π * 2) +0.2102054402283876 sin(2π * 3) +0.10392739760698494 sin(2π * 4) -0.041263270315237216 sin(2π * 5) -0.09997306863049579 sin(2π * 6) -0.06712570420853592 sin(2π * 7) +0.011222936805658968 sin(2π * 8) +0.06647392090929623 sin(2π * 9) +0.0552329180025013 sin(2π * 10)
RMSE: 0.17058677610908896
AIC: 21.907720821138017
5-Fold Cross Validation RMSE: 0.17062371442783827
10-Fold Cross Validation RMSE: 0.17045377317559263
Bootstrap RMSE: 0.17073245970636322
Bias: 0.3434187254885989
Variance: 0.314

Bootstrap RMSE: 0.17008765052465966
Bias: 0.33972689256121674
Variance: 0.3108614390267409
Minimum AIC: -7.965765056236583, m: 2, n: 1
Minimum k-5_Fold Cross Validation RMSE: 0.1371264975800482, m: 10, n: 10
Minimum k-10_Fold Cross Validation RMSE: 0.13653071476227793, m: 10, n: 10
Minimum Bootsrap value: 0.13692956800328107, m: 10, n: 10

